In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [2]:
##Load data

df = pd.read_json("raw_data_shinagawa.json")
df.head()

,Type,Region,MunicipalityCode,Prefecture,Municipality,DistrictName,TradePrice,Area,LandShape,Frontage,...,Breadth,CityPlanning,CoverageRatio,FloorAreaRatio,Period,FloorPlan,Renovation,Remarks,UnitPrice,Purpose
0,Residential Land(Land and Building),Residential Area,13109,Tokyo,Shinagawa Ward,Ebara,47000000,55,Semi-rectangular Shaped,5.0,...,4.1,Quasi-industrial Zone,60.0,300.0,4th quarter 2010,NaN,NaN,NaN,NaN,NaN
1,"Pre-owned Condominiums, etc.",NaN,13109,Tokyo,Shinagawa Ward,Ebara,17000000,20,NaN,NaN,...,NaN,Commercial Zone,80.0,400.0,4th quarter 2010,1K,Not yet,NaN,NaN,NaN
2,Residential Land(Land and Building),Residential Area,13109,Tokyo,Shinagawa Ward,Ebara,130000000,290,Semi-trapezoidal Shaped,12.3,...,4.2,Quasi-industrial Zone,60.0,300.0,3rd quarter 2010,NaN,NaN,Dealings in which auction or arbiter participates,NaN,NaN
3,"Pre-owned Condominiums, etc.",NaN,13109,Tokyo,Shinagawa Ward,Ebara,16000000,20,NaN,NaN,...,NaN,Commercial Zone,80.0,400.0,3rd quarter 2010,1K,Not yet,NaN,NaN,NaN
4,Residential Land(Land and Building),Residential Area,13109,Tokyo,Shinagawa Ward,Ebara,46000000,50,Semi-rectangular Shaped,5.0,...,4.1,Quasi-industrial Zone,60.0,200.0,2nd quarter 2010,NaN,NaN,NaN,NaN,NaN


# Cleaning the data 

In [3]:
##Filter for necessary columns

df2 = df.filter(["Type", "TradePrice", "Area", "FloorPlan"])

In [4]:
def is_float(x):
    try:
        return float(x)  
    except:
        return None

In [5]:
## Remove data points that are not floats

df2["Area"] = df2["Area"].apply(is_float)
df2["TradePrice"] = df2["TradePrice"].apply(is_float)

df3 = df2.dropna(subset=["Area", "TradePrice"])


In [6]:
##Remove Agricultural Land and Forest Land

df4 = df3.copy()
df4.drop(df4[df4['Type'] == "Agricultural Land"].index, inplace = True) 
df4.drop(df4[df4['Type'] == "Forest Land"].index, inplace = True)


In [7]:
type_stats = df4.groupby("Type")["Type"].agg("count")
type_stats

Type
Pre-owned Condominiums, etc.           8769
Residential Land(Land Only)            1836
Residential Land(Land and Building)    2829
Name: Type, dtype: int64

In [8]:
##Create FloorPlan column if necessary

if "FloorPlan" not in df4.columns.values:
    df4["FloorPlan"] = "none"
    
##Fill NaN in FloorPlan

df4["FloorPlan"] = df4["FloorPlan"].fillna(0)

In [9]:
##Drop if not enough data points in FloorPlan

floor_plan_stats = df4.groupby("FloorPlan")["FloorPlan"].agg("count")
floor_plan_less_than_5 = floor_plan_stats[floor_plan_stats < 5]

df4.FloorPlan = df4.FloorPlan.apply(lambda x: None if x in floor_plan_less_than_5 else x)
df5 = df4.dropna(subset=["FloorPlan"])


In [10]:
df5.groupby("FloorPlan")["FloorPlan"].agg("count")

FloorPlan
0                   4957
1DK                  473
1K                  3036
1LDK                 831
1LDK+S                19
1R                   148
2DK                  416
2K                    42
2LDK                1742
2LDK+S                21
3DK                   91
3LDK                1498
4LDK                  78
Open Floor            61
Studio Apartment       8
Name: FloorPlan, dtype: int64

# Removing Outliers

In [11]:
df6 = df5.copy()
df6["price_per_sqm"] = (df5["TradePrice"]/10000) / df5["Area"]
df6.price_per_sqm.describe()

count    13421.000000
mean        83.137175
std         45.859542
min          0.004571
25%         58.181818
50%         78.571429
75%        103.750000
max       1096.774194
Name: price_per_sqm, dtype: float64

In [12]:
##Filter any data points that are greater than one standard deviation per Type

def remove_pps_outliers(df):
    df_out = pd.DataFrame()
    for key, subdf in df.groupby("Type"):
        mean = np.mean(subdf.price_per_sqm)
        stdev = np.std(subdf.price_per_sqm)
        
        reduce_df = subdf[(subdf.price_per_sqm > (mean - stdev)) & (subdf.price_per_sqm <= (mean + stdev))]
        df_out = pd.concat([df_out,reduce_df],ignore_index=True)
        
    return df_out

df7 = remove_pps_outliers(df6)
df7.price_per_sqm.describe()

count    11142.000000
mean        80.518641
std         23.607616
min         24.210526
25%         61.951613
50%         78.947368
75%        100.000000
max        160.000000
Name: price_per_sqm, dtype: float64

# Building Machine Learning Model

In [13]:
##Create dummies for Type and FloorPlan

type_dummies = pd.get_dummies(df7.Type)
floor_plan_dummies = pd.get_dummies(df7.FloorPlan)

type_dummies.head(3)

,"Pre-owned Condominiums, etc.",Residential Land(Land Only),Residential Land(Land and Building)
0,1,0,0
1,1,0,0
2,1,0,0


In [14]:
floor_plan_dummies.head(3)

,0,1DK,1K,1LDK,1LDK+S,1R,2DK,2K,2LDK,2LDK+S,3DK,3LDK,4LDK,Open Floor,Studio Apartment
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [15]:
##Concat dummies

df8 = pd.concat([df7,type_dummies,floor_plan_dummies], axis="columns")
df8.shape

(11142, 23)

In [16]:
X = df8.drop(["TradePrice","price_per_sqm","Type","FloorPlan"],axis="columns")
X

,Area,"Pre-owned Condominiums, etc.",Residential Land(Land Only),Residential Land(Land and Building),0,1DK,1K,1LDK,1LDK+S,1R,2DK,2K,2LDK,2LDK+S,3DK,3LDK,4LDK,Open Floor,Studio Apartment
0,20.0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,20.0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,60.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,60.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,25.0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11137,45.0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11138,45.0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11139,220.0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11140,145.0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
y = df8.TradePrice
y.head()

0    17000000.0
1    16000000.0
2    45000000.0
3    41000000.0
4    16000000.0
Name: TradePrice, dtype: float64

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=10)

In [19]:
lr = LinearRegression()
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

0.7753508060224918